# Bucketing FICO Scores using Dynamic Programming/ KMeans Clustering.

1. We are given a loan dataset with 10k records, with details regarding customers' fico scores, among other details.
2. Goal is to create a function that will dynamically assign fico scores to buckets in such a way that the buckets best summarize the data while also mapping the buckets(boundaries) to ratings where lower rating denotes better fico score.

In [20]:
import pandas as pd
import numpy as np

df=pd.read_csv("Task 3 and 4_Loan_Data.csv")

In [21]:
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [22]:
df.fico_score.nunique()

374

In [23]:
df.fico_score.value_counts()

626    82
653    79
639    77
620    76
636    75
       ..
823     1
804     1
477     1
473     1
794     1
Name: fico_score, Length: 374, dtype: int64

In [25]:
fico_scores=df.fico_score.to_numpy()

In [26]:
fico_scores

array([605, 572, 602, ..., 596, 647, 757], dtype=int64)

#### Pseudo Code

1. Initially defining a certain number of buckets, with equal boundary widths or intervals.
2. Assigning FICO scores to buckets and calculating MSE for FICO scores inside each bucket.
3. Keep Iteratively optimizing the boundaries until they stop moving significantly.

In [53]:
#!! Runs for 15-20minutes+ for 10k records and higher. Complexity O(n^3 * k)

def calculate_mse(bucket):
    """Calculate the Mean Squared Error for a given bucket of scores."""
    if len(bucket) == 0:
        return 0
    mean = np.mean(bucket)
    return np.sum((bucket - mean) ** 2)

def quantize_fico_scores_dp(fico_scores, num_buckets):
    fico_scores = np.sort(fico_scores)
    n = len(fico_scores)

    # DP table: dp[i][j] is the minimum MSE for the first i scores with j buckets
    dp = np.zeros((n + 1, num_buckets + 1))
    cut = np.zeros((n + 1, num_buckets + 1), dtype=int)  # To store cut points

    # Fill the DP table
    for j in range(1, num_buckets + 1):
        for i in range(1, n + 1):
            dp[i][j] = float('inf')
            for k in range(i):  # Consider all previous scores as potential cut points
                mse = calculate_mse(fico_scores[k:i])
                if dp[k][j - 1] + mse < dp[i][j]:
                    dp[i][j] = dp[k][j - 1] + mse
                    cut[i][j] = k

    # Backtrack to find the boundaries
    boundaries = []
    i = n
    for j in range(num_buckets, 0, -1):
        boundaries.append(fico_scores[cut[i][j]:i].mean())
        i = cut[i][j]

    boundaries.reverse()  # Reverse to get boundaries in order
    
    # Append minimum and maximum values to form the full boundary range
    boundaries = [fico_scores[0]] + boundaries + [fico_scores[-1]]

    # Create rating map
    rating_map = {}
    for i in range(num_buckets):
        rating_map[f"{boundaries[i]:.1f} - {boundaries[i + 1]:.1f}"] = num_buckets - i  # Lower rating is better

    return rating_map

# Example usage
# fico_scores = np.random.randint(300, 850, size=100)  # Simulated FICO scores
# fico_scores=df.fico_score.to_numpy()

num_buckets = 3
rating_map = quantize_fico_scores_dp(fico_scores, num_buckets)
print("Rating Map:", rating_map)


Rating Map: {'300.0 - 837.0': 3, '837.0 - 838.0': 2, '838.0 - 849.0': 1}


I tried executing the above Dynamic Programming approach but it results in long running times for 1000+ onwards fico score instances because of multiple loops being run even when we are storing the calculations in a memoization array for reducing redundant calculations. For 10k records the code runs for ~15min which is not feasible at all. The complexity for above code is O(n^3* k)

Using below KMeans clustering approach yield almost instant result even with 100k+ records, the complexity for this approach is O(n). Also, KMeans clustering minimizes sum of squared distances between each buckets, which aligns with our goal of minimizing MSE (mean squared error).

In [51]:
import numpy as np
from sklearn.cluster import KMeans

def quantize_fico_scores_kmeans(fico_scores, num_buckets):
    # Reshaping and sorting fico_scores to a 2D array because KMeans expects 2D data
    fico_scores = np.sort(fico_scores).reshape(-1, 1)
    
    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=num_buckets, random_state=11)
    kmeans.fit(fico_scores)
    
    # Get cluster centers (these are the optimal bucket boundaries)
    centers = np.sort(kmeans.cluster_centers_.flatten())
    
    # Determining boundaries (midpoints between cluster centers)
    boundaries = [(centers[i] + centers[i + 1]) / 2 for i in range(len(centers) - 1)]
    boundaries = [fico_scores.min()] + boundaries + [fico_scores.max()]

    # Create rating map: Assign lower ratings to better scores
    rating_map = {}
    for i in range(len(boundaries) - 1):
        rating_map[f"{boundaries[i]:.0f} - {boundaries[i + 1]:.0f}"] = len(boundaries) - 2 - i #(using integers for boundaries)

    return rating_map

# Example input
# fico_scores = np.random.randint(300, 850, size=1000000)  # Simulated FICO scores
fico_scores=df.fico_score.to_numpy()
num_buckets = 5
rating_map = quantize_fico_scores_kmeans(fico_scores, num_buckets)
print("Rating Map:", rating_map)


Rating Map: {'408 - 558': 4, '558 - 611': 3, '611 - 658': 2, '658 - 709': 1, '709 - 850': 0}


### Conclusion

1. Dynamic programming approach to minimize MSE in buckets lead to a complexity of O(n^3.k)(n being number of fico_scores, k is the number of buckets).
2. KMeans Clustering approach proved to be far more efficient and ran in O(n) complexity.
3. One other method that could be adopted is a mix of Greedy+Dynamic Programming approach. But it leads to complexity of O(n^2* k) as we have to precompute the subarray calculations before hand in a memoization array which in itself costs O(n^2), while the greedy algorithm will then semi-accurately give out best boundaries in a single pass costing O(n)